In [4]:
import json
import time
import datetime
import re
import os
import sys
import subprocess
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import base64
import cv2 as cv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# import easyocr
# from selenium.webdriver.common.action_chains import ActionChains #模拟鼠标

class GetAve:
    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_argument('headless') #无界面
        chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) #防检测
        self.driver = webdriver.Chrome(options=chrome_options)
        self.driver.set_window_size(1920,1080)  #设置分辨率
        
        self.cookies = {
            'domain': 'avedex.cc',
            'expiry': 1662347549,
            'httpOnly': False,
            'name': 'ave_token',
            'path': '/',
            'secure': False,
            'value': '133561fcb642e3599bd1b79f6abd33331661742728595159144'
            }

    def login(self):
        self.driver.get("https://avedex.cc/market")
        time.sleep(3)
        self.driver.add_cookie(cookie_dict = self.cookies)
        self.driver.get("https://avedex.cc/market")
        print('===》正在打开页面')
        time.sleep(5)
        
    def get_coin(self):
        print('===>正在获取中')
        ans = []
        for page in tqdm(range(1,11)):
            # driver.refresh()
            time.sleep(2)
            try:
                page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{page}]'
                self.driver.implicitly_wait(30)  #隐式等待5秒
                coin = self.driver.find_element(by=By.XPATH,value=page_value).text.split('\n')  #代币信息->列表
            except:
                continue  
                
            #跳转到详细页面
            time.sleep(2)
            page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{page}]'
            self.driver.implicitly_wait(30)  #隐式等待5秒
            self.driver.find_element(by=By.XPATH,value=page_value).click()
            
            #点击info
            time.sleep(2)
            try:
                page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[1]/div[1]/div/div[2]'
                self.driver.implicitly_wait(30)  #隐式等待5秒
                self.driver.find_element(by=By.XPATH,value=page_value).click()
            except:
                continue
                
            #获取链接
            time.sleep(2)
            links = []
            try:
                page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li'
                length = len(self.driver.find_elements(by=By.XPATH,value=page_value))
                page_value = f'/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li[{length}]/div/a'
                for i in self.driver.find_elements(by=By.XPATH,value=page_value):
                    link = i.get_attribute('href')
                    links.append(link)
            except Exception as e:
                self.driver.get_screenshot_as_file('error截图.png')
                print(e)
                continue
                
            coin.append(links)  
            ans.append(coin)
            
            #代币合约&所属链
            current_url = self.driver.current_url
            contract = re.search('(?<=token/).*?(?=-)',current_url).group()
            chain = re.search('(?<=-).*',current_url).group()
            coin.extend([contract,chain])

            url_now = self.driver.current_url
            while url_now != 'https://avedex.cc/market':
                self.driver.back() #返回上一级
                time.sleep(2)
                url_now = self.driver.current_url
                
        date_ = datetime.datetime.now().strftime('%Y-%m-%d %H')
        df = pd.DataFrame(ans,columns=['序号','币种','最新价格','24h涨幅','流通量','开盘价','风险性','相关链接','合约地址','所属链'])
        df['更新时间'] = date_
        df = df[['序号','更新时间','币种','最新价格','24h涨幅','流通量','开盘价','风险性','相关链接','合约地址','所属链']]
        print(f'获取完成,更新时间:{date_}')
        
        return df
    

GA = GetAve()
GA.login()
test = GA.get_coin()

===》正在打开页面
===>正在获取中


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:20<00:00, 26.09s/it]

获取完成,更新时间:2022-08-29 11


In [5]:
test

,序号,更新时间,币种,最新价格,24h涨幅,流通量,开盘价,风险性,相关链接,合约地址,所属链
0,1,2022-08-29 11,QJ,101.511,-2.14%,"8,680.3",123.461,55,[],0x6743665257c429605aa344ed04dff7a183a50296,bsc
1,2,2022-08-29 11,ENFT,1.355,-3.12%,"11,948,797.726",--,55,"[http://twitter.com/eNFT111, https://t.me/enft...",0xf6a3d5c35e5641553bce157cc0e8719f81d56bab,bsc
2,3,2022-08-29 11,FGD,4.804,+0.91%,"19,476,800",0.156,20,"[https://twitter.com/FreedomgodDao, https://t....",0x0566b9a8ffb8908682796751eed00722da967be0,bsc
3,4,2022-08-29 11,BSC-HO,2.079,+0.33%,"20,039,155.766",--,20,[],0x41515885251e724233c6ca94530d6dcf3a20dec7,bsc
4,5,2022-08-29 11,SFC,0.0164,+1.47%,"323,682,036.867",--,20,[],0x8d9cad20080110998966b63a87fb9d51eb49b798,bsc
5,6,2022-08-29 11,KYOTO,8.874,+1.81%,"1,363,296.932",19.49,55,[],0x69104fb28f4bb9f6efc899bd1d94f386cdf1b9da,bsc
6,7,2022-08-29 11,DRAC,0.0206,+0.0%,"93,318,954.58",0.00596,55,[https://t.me/TeddyDoge_Chinese],0x123458c167a371250d325bd8b1fff12c8af692a7,bsc
7,8,2022-08-29 11,BSW,0.299,+0.82%,"350,426,870.56",--,55,"[https://twitter.com/Biswap_Dex, https://t.me/...",0x965f527d9159dce6288a2219db51fc6eef120dd1,bsc
8,9,2022-08-29 11,KFC,26.569,-4.69%,"300,462.09",21.118,55,[],0xf6ad779ad69fee7e44514e3efd20090ab43fbaef,bsc
9,10,2022-08-29 11,CEEK,0.168,-0.43%,"344,872,379.972",--,55,"[https://twitter.com/ceek, https://t.me/ceekvr...",0xe0f94ac5462997d2bc57287ac3a3ae4c31345d66,bsc


In [ ]:
import json
import time
import datetime
import re
import os
import sys
import subprocess
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import base64
import cv2 as cv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import easyocr
# from selenium.webdriver.common.action_chains import ActionChains #模拟鼠标

chrome_options = Options()
chrome_options.add_argument('headless') #无界面
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) #防检测
driver = webdriver.Chrome(options=chrome_options)
# driver.maximize_window()  #最大化窗口
driver.set_window_size(1920,1080)  #设置分辨率
driver.get("https://avedex.cc/market")
print('===》正在打开页面')
time.sleep(5)

#验证码截图
# driver.get_screenshot_as_file('截图.png')
page_value = '/html/body/div[1]/div[3]/div/div/div[2]/form/div[1]/img'
img = driver.find_element(by=By.XPATH,value=page_value)
head = base64.b64decode(img.get_attribute('src')[22:])
with open('pic.png','wb') as f:         
    f.write(head)
    
#====>输入验证码
page_value = '/html/body/div[1]/div[3]/div/div/div[1]/span'
driver.implicitly_wait(30)  #隐式等待10秒
vertify = driver.find_element(by=By.XPATH,value=page_value).text
while vertify == 'Security Verification': #验证码错误则重新输入验证码
    page_value = "/html/body/div[1]/div[3]/div/div/div[2]/form/div[2]/div[2]/div/input"
    driver.implicitly_wait(30)  #隐式等待10秒
    driver.find_element(by=By.XPATH,value=page_value).send_keys(input("请输入验证码"))
    # #点击验证
    page_value = "/html/body/div[1]/div[3]/div/div/div[2]/form/div[3]/div[2]/button"
    driver.find_element(by=By.XPATH,value=page_value).click()
    time.sleep(2)
    page_value = '/html/body/div[1]/div[3]/div/div/div[1]/span'
    driver.find_element(by=By.XPATH,value=page_value).text
    vertify = driver.find_element(by=By.XPATH,value=page_value).text
    
# 排名前10热度的代币
print('===>正在获取中')
# ans = {'序号':[],'币种':[],'最新价格':[],'24h涨幅':[],'流通量':[],'开盘价':[],'流通性':[],'电报':[],'推特':[],'其它链接':[]}
ans = []
for page in tqdm(range(1,11)):
    # driver.refresh()
    time.sleep(2)
    try:
        page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{page}]'
        driver.implicitly_wait(30)  #隐式等待5秒
        coin = driver.find_element(by=By.XPATH,value=page_value).text.split('\n')
    except:
        continue  
        
    #跳转到详细页面
    time.sleep(2)
    page_value = f'/html/body/div[1]/main/section/div[2]/div/div[1]/div[3]/div/div[1]/div/table/tbody/tr[{page}]'
    driver.implicitly_wait(30)  #隐式等待5秒
    driver.find_element(by=By.XPATH,value=page_value).click()
    # time.sleep(2)
    # print(driver.current_url)
    # driver.back()
    
    #点击info
    time.sleep(2)
    try:
        page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[1]/div[1]/div/div[2]'
        driver.implicitly_wait(30)  #隐式等待5秒
        driver.find_element(by=By.XPATH,value=page_value).click()
    except:
        continue
        
    #info
    # page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul'
    # driver.implicitly_wait(30)  #隐式等待5秒
    # driver.find_element(by=By.XPATH,value=page_value).text
    
    #获取链接
    time.sleep(2)
    links = []
    try:
        page_value = '/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li'
        length = len(driver.find_elements(by=By.XPATH,value=page_value))
        page_value = f'/html/body/div[1]/main/section/div[1]/div[4]/section/div[2]/div[1]/div/ul/li[{length}]/div/a'
        for i in driver.find_elements(by=By.XPATH,value=page_value):
            link = i.get_attribute('href')
            links.append(link)
    except Exception as e:
        driver.get_screenshot_as_file('error截图.png')
        print(e)
        continue
    
    coin.append(links)  
    ans.append(coin)
    
    #代币合约&所属链
    current_url = driver.current_url
    contract = str(re.search('(?<=token/).*?(?=-)',current_url).group())
    chain = re.search('(?<=-).*',current_url).group()
    coin.extend([contract,chain])
    
    url_now = driver.current_url
    while url_now != 'https://avedex.cc/market':
        driver.back() #返回上一级
        time.sleep(2)
        url_now = driver.current_url
    
date_ = datetime.datetime.now().strftime('%Y-%m-%d %H')  
df = pd.DataFrame(ans,columns=['序号','币种','最新价格','24h涨幅','流通量','开盘价','风险性','相关链接','合约地址','所属链'])
df['更新时间'] = date_
df = df[['序号','更新时间','币种','最新价格','24h涨幅','流通量','开盘价','风险性','相关链接','合约地址','所属链']]

print(f'获取完成,更新时间:{date_}')